In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Cài nnU-Net bản chuẩn từ PyPI (Nhanh và sạch)
!pip install nnunetv2

# Cài thêm các thư viện bổ trợ nếu thiếu
!pip install nibabel matplotlib medpy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 kB 9.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 111.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 10.4 MB/s eta 0:00:00
  Created wheel for nnunetv2: filename=nnunetv2-2.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.3/156.3 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for medpy: filename=MedPy-0.5.2-py3-none-any.whl size=224710 sha256=8e8023b9b0bc915fe73f860e6fb25ee27f8fdb5f0577457699d6880e0fcfda74
  Stored in directory: /root/.cache/pip/wheels/89/5a/f8/b3def53b9c2133d2f8698ea2173bb5df63bd8e761ce8e9aec9
Successfully built medpy


## 1. Giải nén foler input

In [ ]:
!ls

drive  sample_data


### a. Giải nén `nnUNet_preprocessed` vào thư mục content của Colab

In [ ]:
import os
import zipfile
from tqdm import tqdm
import shutil

In [ ]:
# GIẢI NÉN DỮ LIỆU PREPROCESSED
# ----------------------------------------------------------------------

os.environ['nnUNet_raw'] = "/content/nnUNet_raw"
os.environ['nnUNet_preprocessed'] = "/content/nnUNet_preprocessed"
os.environ['nnUNet_results'] = "/content/drive/MyDrive/XUM_project/nnUNet_results"

!mkdir -p $nnUNet_raw
!mkdir -p $nnUNet_preprocessed

zip_path_drive = "/content/drive/MyDrive/XUM_project/nnUNET_preprocessed.zip"
zip_local = "/content/nnUNET_preprocessed.zip"
extract_root = "/content/nnUNet_preprocessed"

print("📦 Copy file từ Drive…")
!cp "$zip_path_drive" "$zip_local"

print("💨 Copy xong. Bắt đầu giải nén…")

with zipfile.ZipFile(zip_local, 'r') as zip_ref:
    # Detect folder root name inside zip (vd: nnUNET_preprocessed/)
    root_name = zip_ref.namelist()[0].split('/')[0]

    file_list = zip_ref.infolist()
    total_files = len(file_list)

    print(f"🔧 Phát hiện folder root trong ZIP: {root_name}")
    print("🔧 Đang giải nén, loại bỏ lớp folder đầu tiên…")

    for file in tqdm(file_list, total=total_files, desc="🔄 Unzipping"):
        file_path = file.filename

        # Bỏ lớp folder đầu tiên
        fixed_path = file_path.replace(root_name + "/", "", 1)

        if fixed_path.strip() == "":
            continue

        # Chỉ định nơi lưu đúng
        target_path = os.path.join(extract_root, fixed_path)

        # Tạo thư mục nếu cần
        if file.is_dir():
            os.makedirs(target_path, exist_ok=True)
        else:
            os.makedirs(os.path.dirname(target_path), exist_ok=True)
            with zip_ref.open(file) as source, open(target_path, "wb") as target:
                target.write(source.read())

os.remove(zip_local)

print("🎉 GIẢI NÉN XONG VÀ ĐÃ TỰ ĐỘNG FIX FOLDER DOUBLE-LAYER! 😎🔥")

📦 Copy file từ Drive…
💨 Copy xong. Bắt đầu giải nén…
🔧 Phát hiện folder root trong ZIP: nnUNET_preprocessed
🔧 Đang giải nén, loại bỏ lớp folder đầu tiên…


🔄 Unzipping: 100%|██████████| 2585/2585 [02:45<00:00, 15.62it/s]


🎉 GIẢI NÉN XONG VÀ ĐÃ TỰ ĐỘNG FIX FOLDER DOUBLE-LAYER! 😎🔥


### b. Giải nén `nnUNet_raw` vào thư mục content của Colab

In [ ]:
# GIẢI NÉN DỮ LIỆU RAW (QUAN TRỌNG CHO INFERENCE)
# ----------------------------------------------------------------------

# 1. Thiết lập đường dẫn (Đảm bảo biến môi trường vẫn đúng)
os.environ['nnUNet_raw'] = "/content/nnUNet_raw"
!mkdir -p $nnUNet_raw

# Đường dẫn file zip trên Drive (Bạn kiểm tra lại tên file cho đúng nhé)
# Giả sử bạn lưu là nnUNet_raw.zip trong thư mục XUM_project
zip_path_drive = "/content/drive/MyDrive/XUM_project/nnUNET_raw.zip"
zip_local = "/content/nnUNet_raw.zip"
extract_root = "/content/nnUNet_raw"

if not os.path.exists(zip_path_drive):
    print(f"❌ Lỗi: Không tìm thấy file {zip_path_drive}")
    print("👉 Hãy kiểm tra lại tên file zip raw trên Google Drive của bạn.")
else:
    print("📦 Copy file Raw Data từ Drive...")
    !cp "$zip_path_drive" "$zip_local"

    print("💨 Copy xong. Bắt đầu giải nén Raw Data...")

    with zipfile.ZipFile(zip_local, 'r') as zip_ref:
        # Detect folder root name inside zip (ví dụ: nnUNet_raw/)
        # Để tránh bị lồng thư mục kiểu /content/nnUNet_raw/nnUNet_raw/Dataset...
        root_name = zip_ref.namelist()[0].split('/')[0]

        file_list = zip_ref.infolist()
        total_files = len(file_list)

        print(f"🔧 Phát hiện folder root trong ZIP: {root_name}")
        print("🔧 Đang giải nén và loại bỏ lớp folder thừa...")

        for file in tqdm(file_list, total=total_files, desc="🔄 Unzipping Raw"):
            file_path = file.filename

            # Logic: Nếu file nằm trong folder root, ta bỏ tên root đi để file nhảy ra ngoài
            # Ví dụ: nnUNet_raw/Dataset101/abc.nii -> Dataset101/abc.nii
            if file_path.startswith(root_name + "/"):
                fixed_path = file_path.replace(root_name + "/", "", 1)
            else:
                fixed_path = file_path # Nếu nén không có folder root thì giữ nguyên

            if fixed_path.strip() == "":
                continue

            # Chỉ định nơi lưu đúng
            target_path = os.path.join(extract_root, fixed_path)

            # Tạo thư mục nếu cần
            if file.is_dir():
                os.makedirs(target_path, exist_ok=True)
            else:
                os.makedirs(os.path.dirname(target_path), exist_ok=True)
                with zip_ref.open(file) as source, open(target_path, "wb") as target:
                    target.write(source.read())

    os.remove(zip_local)
    print("🎉 ĐÃ GIẢI NÉN RAW DATA THÀNH CÔNG! SẴN SÀNG CHẠY INFERENCE! 🚀")

📦 Copy file Raw Data từ Drive...
💨 Copy xong. Bắt đầu giải nén Raw Data...
🔧 Phát hiện folder root trong ZIP: nnUNET_raw
🔧 Đang giải nén và loại bỏ lớp folder thừa...


🔄 Unzipping Raw: 100%|██████████| 1845/1845 [03:57<00:00,  7.77it/s]

🎉 ĐÃ GIẢI NÉN RAW DATA THÀNH CÔNG! SẴN SÀNG CHẠY INFERENCE! 🚀


## 2. Inference

In [ ]:
# ------------------------------------------------------------------
# 🧠 CHẾ ĐỘ 1: CHẠY MODEL EDL (UNCERTAINTY DECOMPOSITION)
# ------------------------------------------------------------------
# Script này sẽ thực hiện quy trình full:
# 1. Tự động "tiêm" (inject) file EDLTrainer.py vào thư viện nnU-Net hệ thống.
# 2. Load trọng số Model EDL (checkpoint_best.pth) từ đường dẫn config 'edl'.
# 3. Chạy Inference và tính toán toán học để tách Uncertainty thành:
#    - Aleatoric (Nhiễu dữ liệu)
#    - Epistemic (Mô hình chưa biết)
# 4. Lưu kết quả Segmentation + 3 bản đồ Uncertainty (.nii.gz).
!python "/content/drive/MyDrive/XUM_project/src/main.py" --mode edl

🏁 --- STARTING PIPELINE | MODE: EDL | FOLD: 0 ---
🔧 Initializing Engine | Mode: EDL...
🚀 Initializing nnU-Net Predictor...
📂 Model loaded from: /content/drive/MyDrive/XUM_project/nnUNet_results/Dataset101_BraTS2020/EDLTrainer__nnUNetPlans__3d_fullres
📂 Đã load danh sách Validation Fold 0: 74 ca.
⚙️ Mode: VALIDATION SPLIT -> Found 74 cases (Filtered).

Index    | Case ID         | Dice WT  | Dice TC  | Dice ET  | Mean    
-------------------------------------------------------------------------------------
100% 8/8 [00:16<00:00,  2.10s/it]
1        | BRATS_011       | 0.9184   | 0.9098   | 0.7531   | 0.8604
    📸 Drawing Slice: 111
    ✅ Saved Snapshot: /content/drive/MyDrive/XUM_project/inference_results_edl/BRATS_011_slice111_viz.png
100% 8/8 [00:16<00:00,  2.02s/it]
2        | BRATS_012       | 0.8588   | 0.9274   | 0.8367   | 0.8743
    📸 Drawing Slice: 119
    ✅ Saved Snapshot: /content/drive/MyDrive/XUM_project/inference_results_edl/BRATS_012_slice119_viz.png
100% 8/8 [00:16<00:00

In [ ]:
# Test thử trên 5 ca đầu tiên cho nhanh
!python /content/drive/MyDrive/XUM_project/src/analysis_ause.py --mode edl

🚀 BẮT ĐẦU PHÂN TÍCH CHUẨN RESEARCH (V9) | MODE: EDL
🔍 Found 74 cases.

📊 Analyzing Class: WT ...
Processing WT: 100% 74/74 [01:11<00:00,  1.04it/s]
/content/drive/MyDrive/XUM_project/src/analysis_ause.py:252: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  ause_total = np.trapz(mean_opt - mean_dice, dx=1.0/len(valid_retention))
/content/drive/MyDrive/XUM_project/src/analysis_ause.py:257: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  ause_alea = np.trapz(mean_opt - mean_alea, dx=1.0/len(valid_retention))
/content/drive/MyDrive/XUM_project/src/analysis_ause.py:262: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  ause_epis = np.trapz(mean_opt - mean_epis, dx=1.0/len(valid_retention))

📊 Analyzing Class: TC ...
Processing 